- 2020-08-10
- OS: Windows 10
- yolov4: 1.1.0

In [1]:
from yolov4.tf import YOLOv4

yolo = YOLOv4()

yolo.classes = "coco.names"

yolo.input_size = 608
yolo.batch_size = 2

In [2]:
import time

start_t = time.time()
dataset = yolo.load_dataset("train2017.txt", image_path_prefix="D:/coco/train2017")
print(time.time() - start_t)

3.732321262359619


In [3]:
import tensorflow as tf
import numpy as np
import cv2

for i, (images, gt) in enumerate(dataset):
    for j in range(len(images)):
        _candidates = []
        for candidate in gt:
            grid_size = candidate.shape[1]
            _candidates.append(
                tf.reshape(candidate[j], shape=(1, grid_size * grid_size * 3, -1))
            )
        candidates = np.concatenate(_candidates, axis=1)

        frame = images[j, ...] * 255
        frame = frame.astype(np.uint8)

        pred_bboxes = yolo.candidates_to_pred_bboxes(candidates[0])
        pred_bboxes = yolo.fit_pred_bboxes_to_original(pred_bboxes, frame.shape)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        image = yolo.draw_bboxes(frame, pred_bboxes)
        cv2.namedWindow("result", cv2.WINDOW_AUTOSIZE)
        cv2.imshow("result", image)
        while cv2.waitKey(10) & 0xFF != ord("q"):
            pass
    if i == 10:
        break

cv2.destroyWindow("result")